# Análise de sentimentos na base do IMDB usando BoW e LORA

Leandro Carísio Fernandes

# Parâmetros

In [ ]:
MAX_VOCAB_SIZE = 20000  # Máximo de palavras no vocabulário
VOCAB_SIZE = -1         # Será ajustado automaticamente depois do cálculo do vocabulário

HIDDEN_SIZE = 200       # Tamanho da camada oculta

LR = 0.1                # Learning rate
NUM_EPOCHS = 7          # Número de épocas para rodar o treinamento completo. Após 7 épocas já não tem melhoras significativas no dataset de validação
NUM_EPOCHS_LORA = 10    # Número de épocas para rodar fine-tuning com LoRA (será executado após apenas uma época de treinamento completo)
RANK = 1                # Rank que será usado. Quanto menor, menos parâmetros serão treinados no fine-tuning
ALPHA = 1               # Usado pro cálculo do fator de escala na matriz delta_w. Escala = alpha/rank

In [ ]:
import torch

# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('using CPU')

GPU: Tesla T4


# Seed

In [ ]:
import random
import torch.nn.functional as F
import numpy as np

def inicializa_seeds():
  random.seed(123)
  np.random.seed(123)
  torch.manual_seed(123)

inicializa_seeds()

## Download dos dados

In [ ]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz
!tar -xzf aclImdb.tgz

--2024-04-12 13:31:57--  http://files.fast.ai/data/aclImdb.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.3.19, 172.67.69.159, 104.26.2.19, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/aclImdb.tgz [following]
--2024-04-12 13:31:57--  https://files.fast.ai/data/aclImdb.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.3.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145982645 (139M) [application/x-gtar-compressed]
Saving to: ‘aclImdb.tgz’

aclImdb.tgz         100%[===================>] 139.22M  98.4MB/s    in 1.4s    

2024-04-12 13:31:59 (98.4 MB/s) - ‘aclImdb.tgz’ saved [145982645/145982645]



## Separação em treino/validação/teste

In [ ]:
import os

max_valid = 5000

def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:-max_valid]
y_train = y_train[:-max_valid]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('\n3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('\n3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('\n3 primeiras amostras validação:')
for x, y in zip(x_test[:3], y_test[:3]):    # No código original, estava x_valid
    print(y, x[:100])

print('\n3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.

3 primeiras amostras treino:
False Some of the filmmakers who are participating in this series have made some really great films but th
False The original Boogeyman was a silly but entertaining supernatural slasher flick. It was by no means a
True This tender beautifully crafted production delved deep down bitter sweet into my being. The irrevere

3 últimas amostras treino:
False Bathebo, you big dope.<br /><br />This is the WORST piece of crap I've seen in a long time. I have j
True They filmed this movie out on long Island, where I grew up. My brother and his girlfriend were extra
True I am amazed that movies like this can still be made. I watch all kinds of movies all the time with m

3 primeiras amostras validação:
True The Book of Life was rather like a short snack, whetting the appetite for Hartley's next full length
True In the ever growing film genre of comic book adaptations, Blade is by far 

# Tokenizador e encoder

In [ ]:
## Função gerada com ajuda do ChatGPT
import re

def tokenizar(frase):
    # return frase.split() # Usar esse return para uma tokenização simples (apenas split)
    # Converter a frase para minúsculo
    frase = frase.lower()
    # Remover caracteres especiais e pontuação usando expressões regulares
    frase = re.sub(r'[^\w\s]', '', frase)
    # Dividir a frase em palavras
    palavras = frase.split()
    return palavras

# Exemplo de uso
frase = "Olá! Como você está? Eu estou bem, obrigado."
palavras = tokenizar(frase)
print(palavras)

['olá', 'como', 'você', 'está', 'eu', 'estou', 'bem', 'obrigado']


In [ ]:
from collections import Counter

idx_amostras = list(range(len(x_train)))

counter_palavras = Counter()
for review in x_train:
    # Com tokenizador:
    counter_palavras.update(tokenizar(review))

# Cria um vocabulário com os tokens mais frequentes
most_frequent_words = ["<UNK>"] + sorted(counter_palavras, key=counter_palavras.get, reverse=True)[:MAX_VOCAB_SIZE-1]
vocab = {word: i for i, word in enumerate(most_frequent_words)} # word é indexado de 0 (UNK) até MAX_VOCAB_SIZE
VOCAB_SIZE = len(vocab.keys()) # Se MAX_VOCAB_SIZE for muito grande, é possível que o vocabulário não tenha palavras suficientes. Então ajusta pro tamanho correto

In [ ]:
most_frequent_words[0:10]

['<UNK>', 'the', 'a', 'and', 'of', 'to', 'is', 'in', 'it', 'i']

In [ ]:
def encode_sentence(sentence, vocab):
  return [vocab.get(word, 0) for word in tokenizar(sentence)] # 0 for OOV

encode_sentence("I like Pizza PALAVRAINEXISTENTE", vocab)

[9, 38, 8027, 0]

# Definição da classe Dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
from typing import List

class ImdbDataset(Dataset):
  def __init__(self, x_data: List[str], y_data: List[bool], vocab) -> None:
    self.vocab = vocab

    tamanho_vocab = len(vocab.keys()) # isso dá igual VOCAB_SIZE. Pra não pegar da variável global

    # Como as bases cabem em memória, faz cache
    self.x = []
    for review in x_data:
        x = torch.zeros(tamanho_vocab)
        x[encode_sentence(review, self.vocab)] = 1
        self.x.append(x)

    self.y = None if y_data is None else [torch.tensor(1 if y_is_true else 0) for y_is_true in y_data]

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
# Checa a implementação do dataset:

x_temp, y_temp = x_train[:10], y_train[0:10]
dataset_temp = ImdbDataset(x_temp, y_temp, vocab)

for i in range(len(x_temp)):
  encoded_x = encode_sentence(x_temp[i], vocab)
  x = torch.zeros(VOCAB_SIZE)
  x[encoded_x] = 1
  print(torch.all(x == dataset_temp[i][0]))
  print(torch.tensor(y_temp[i]) == dataset_temp[i][1])


tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


# DataLoader

In [ ]:
%%time
batch_size = 128
# define dataloaders
train_data = ImdbDataset(x_train, y_train, vocab)
test_data = ImdbDataset(x_test, y_test, vocab)
val_data = ImdbDataset(x_valid, y_valid, vocab)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

CPU times: user 8.43 s, sys: 2.19 s, total: 10.6 s
Wall time: 10.7 s


# Implementação camada LoRA

In [ ]:
import torch
import torch.nn as nn
import math

class LoRALayer(nn.Module):
  def inicializa_pesos_LoRA(self):
    # Parâmetros LoRA
    self.A = nn.Parameter(torch.Tensor(self.output_dim, self.rank), requires_grad=self.use_lora)
    self.B = nn.Parameter(torch.Tensor(self.rank, self.input_dim), requires_grad=self.use_lora)

    nn.init.normal_(self.A, mean=0.0)
    nn.init.zeros_(self.B)

  def __init__(self, input_dim, output_dim, rank, alpha=1.0, use_lora=False):
    super(LoRALayer, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.rank = rank
    self.alpha = alpha
    self.use_lora = use_lora

    self.scale = self.alpha/self.rank

    # Parâmetros originais da camada linear
    # Note que o nn.Parameter está no formato output_dim x input_dim,
    # e não input_dim x output_dim como era de se esperar
    # Isso é feito pois o PyTorch implementa os pesos da camada linear
    # dessa forma. Em vez de calcular x*A + b, ele faz x * A_T + b,
    # onde _T é a transposta. Por isso é necessário inverter isso
    self.weight = nn.Parameter(torch.Tensor(output_dim, input_dim))
    self.bias = nn.Parameter(torch.Tensor(output_dim))

    # Inicialização
    nn.init.xavier_normal_(self.weight)
    nn.init.uniform_(self.bias, 0, 0)
    self.inicializa_pesos_LoRA()

  def forward(self, x):
    # Se estiver usando LoRA, é necessário considerar os pesos modificados
    if self.use_lora:
      delta_w = self.scale * (self.A @ self.B)
      modified_weight = self.weight + delta_w
      return nn.functional.linear(x, modified_weight, self.bias)
    else:
      # Caso contrário, usa os pesos originais
      return nn.functional.linear(x, self.weight, self.bias)
      # Obs.: O controle dos gradientes é feito dentro do toggle_lora


  def toggle_lora(self, use_lora=False):
    self.use_lora = use_lora
    # Desabilita os gradientes de weight e bias se estiver usando a camada LoRA
    self.weight.requires_grad = not self.use_lora
    self.bias.requires_grad = not self.use_lora
    # Habilita os gradientes de A e B se estiver usando a camada LoRA
    self.A.requires_grad = self.use_lora
    self.B.requires_grad = self.use_lora

  def desabilita_lora_e_transfere_para_modelo(self):
    # A ideia desse método é desabilitar o LoRA e alterar o peso original
    # para considerar o treinamento usando LoRA
    delta_w = self.scale * self.A @ self.B
    self.weight.data += delta_w.data

    self.toggle_lora(False)
    self.inicializa_pesos_LoRA()

In [ ]:
# Testando as dimensões
# Pega um batch e passa o primeiro conjunto de inputs numa camada LoRA ativada:
batch_teste = next(iter(test_loader))
camada_lora = LoRALayer(VOCAB_SIZE, HIDDEN_SIZE, rank=10, use_lora=True)
o1 = camada_lora(batch_teste[0][0])
print('Cria uma camada LoRA e ativa o uso da camada (B é inicializado com zeros)')
print('Faz o forward usando a camada => o1\n')

# Desativa a camada LoRA e passa de novo o mesmo input:
camada_lora.toggle_lora(False)
o2 = camada_lora(batch_teste[0][0])
print('Desativa o uso da camada LoRA (faz a conta só com os pesos normais)')
print('Faz o forward => o2')
print(f'Testa de o1 == o2. Esperado: True. Resultado: {torch.all(o1 == o2)}\n')

# Na inicialização, B é tudo zero.
# Altera diretamente o B com dados aleatórios para que a camada LoRA mude o resultado
# Entretanto, mantém a camada LoRA desativada
nn.init.normal_(camada_lora.B, mean=0.0)
camada_lora.toggle_lora(False)
o3 = camada_lora(batch_teste[0][0])
print('Altera B para algum valor aleatório e mantém a camada LoRA desativada')
print('Faz o forward => o3')
print(f'Testa de o2 == o3. Esperado: True. Resultado: {torch.all(o2 == o3)}\n')

# Agora abilita a camada LoRA. Mantém B populado com algum valor
camada_lora.toggle_lora(True)
o4 = camada_lora(batch_teste[0][0])
print('Mantém B com valores e ativa a camada LoRA')
print('Faz o forward => o4')
print(f'Testa de o2 == o4. Esperado: False. Resultado: {torch.all(o2 == o4)}\n')

# Agora desabilita a camada LoRA transferindo os resultados para os pesos originais
# e calcula novamente
camada_lora.desabilita_lora_e_transfere_para_modelo()
o5 = camada_lora(batch_teste[0][0])
print('Desabilita a camada LoRA, transferindo os pesos para a camada linear')
print('Faz o forward => o5')
print(f'Testa de o4 == o5. Esperado: True. Resultado: {torch.all(o4 == o5)}. Dif máxima: {torch.max((o4-o5).abs())}\n')

# Gera qualquer coisa na camada LoRA, mas não habilita ela. Não tem que fazer diferença
nn.init.normal_(camada_lora.B, mean=0)
o6 = camada_lora(batch_teste[0][0])
print('Gera qualquer coisa na camada LoRA, mas não habilita ela')
print('Faz o forward => o6')
print(f'Testa de o5 == o6. Esperado: True. Resultado: {torch.all(o5 == o6)}\n')

# Agora habilita a camada LoRA sem inicializá-la. Tem lixo lá por conta da
# inicialização dos pesos de B com dados (teste anterior)
camada_lora.toggle_lora(True)
o7 = camada_lora(batch_teste[0][0])
print('Habilita a camada LoRA mantendo lixo em B')
print('Faz o forward => o7')
print(f'Testa de o6 == o7. Esperado: False. Resultado: {torch.all(o6 == o7)}\n')

Cria uma camada LoRA e ativa o uso da camada (B é inicializado com zeros)
Faz o forward usando a camada => o1

Desativa o uso da camada LoRA (faz a conta só com os pesos normais)
Faz o forward => o2
Testa de o1 == o2. Esperado: True. Resultado: True

Altera B para algum valor aleatório e mantém a camada LoRA desativada
Faz o forward => o3
Testa de o2 == o3. Esperado: True. Resultado: True

Mantém B com valores e ativa a camada LoRA
Faz o forward => o4
Testa de o2 == o4. Esperado: False. Resultado: False

Desabilita a camada LoRA, transferindo os pesos para a camada linear
Faz o forward => o5
Testa de o4 == o5. Esperado: True. Resultado: False. Dif máxima: 9.5367431640625e-07

Gera qualquer coisa na camada LoRA, mas não habilita ela
Faz o forward => o6
Testa de o5 == o6. Esperado: True. Resultado: True

Habilita a camada LoRA mantendo lixo em B
Faz o forward => o7
Testa de o6 == o7. Esperado: False. Resultado: False



# Modelo

In [ ]:
import torch.nn as nn
import torch.optim as optim

class OneHotMLP(nn.Module):
  def __init__(self, vocab_size, hidden_size, rank, alpha, n_logitos=2):
    super(OneHotMLP, self).__init__()

    self.n_logitos = n_logitos

    #self.fc1 = nn.Linear(vocab_size, hidden_size)
    # No início do treinamento é feito sem o uso da camada LoRA, por isso começa com use_lora=False
    self.fc1 = LoRALayer(vocab_size, hidden_size, rank, alpha, use_lora=False)
    self.fc2 = nn.Linear(hidden_size, n_logitos)

    self.relu = nn.ReLU()

  def forward(self, x):
    # x tem tamanho [B, VOCAB]

    # Após a primeira camada, o tamanho será [B, HIDDEN_SIZE]
    o = self.fc1(x)

    # Após ReLU, o tamanho continua [B, HIDDEN_SIZE]
    o = self.relu(o)

    # Após a segunda camada, o tamanho será [B, n_logitos]
    o = self.fc2(o)

    return o

  def desabilita_lora_e_transfere_para_modelo(self):
    self.fc1.desabilita_lora_e_transfere_para_modelo()

  def toggle_lora(self, use_lora):
    self.fc1.toggle_lora(use_lora)


# Laço de treinamento

In [ ]:
def calcula_loss_e_acuracia(model, loader):
  criterion = nn.CrossEntropyLoss()

  # Garante que nenhum gradiente seja calculado
  with torch.no_grad():
    # Coloca o modelo no modo de avaliação (não treinamento)
    model.eval()

    total_loss = 0.0
    total_samples = 0
    total_predicoes_corretas = 0.0

    for inputs, labels in loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = model(inputs)
      # Calcula a perda
      probabilities = nn.functional.softmax(outputs, dim=1)
      loss = criterion(probabilities, labels)
      # Conta as predições corretas (para o cálculo da acurácia)
      _, predicoes = torch.max(probabilities, 1)
      total_predicoes_corretas += (predicoes == labels).sum().item()

      # Acumula a perda e o número total de amostras
      total_loss += loss.item() * inputs.size(0)
      total_samples += inputs.size(0)

  loss = total_loss / total_samples
  acuracia = total_predicoes_corretas / total_samples

  return loss, acuracia

def print_loss_acuracia(msg, model, loader):
  loss, acc = calcula_loss_e_acuracia(model, loader)
  print(f"{msg}. Loss: {loss}. Acc: {acc}")
  return loss, acc

In [ ]:
# Testa o cálculo da loss em um dataloader. Ele deve ser aproximadamente ln(2):
import math

# Model instantiation
model = OneHotMLP(VOCAB_SIZE, HIDDEN_SIZE, RANK, ALPHA, 2).to(device)

print(f"Valor esperado da loss: {math.log(2)}")
print_loss_acuracia("Antes de treinar o modelo [VAL]: ", model, val_loader)

Valor esperado da loss: 0.6931471805599453
Antes de treinar o modelo [VAL]: . Loss: 0.692955906677246. Acc: 0.5194


(0.692955906677246, 0.5194)

In [ ]:
temp1 = list(filter(lambda p: p.requires_grad, model.parameters()))
temp2 = list(filter(lambda p: True, model.parameters()))

In [ ]:
from tqdm import tqdm
import time

def train_model(model, lr=LR, num_epochs=NUM_EPOCHS):
  model = model.to(device)
  # optimizer = optim.SGD(model.parameters(), lr=lr)
  # Usa só os parâmetros que devem ser atualizados
  optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
  criterion = nn.CrossEntropyLoss()

  total_parametros_treinaveis = sum(p.numel() for p in model.parameters() if p.requires_grad)
  total_parametros = sum(p.numel() for p in model.parameters())
  print(f'Total de parâmetros treináveis: {total_parametros_treinaveis}/{total_parametros}')
  print_loss_acuracia("[TRAIN] Antes de iniciar o treinamento", model, train_loader)
  # Training loop
  total_tempo_treinamento = 0
  for epoch in range(1, num_epochs+1):
    print(f'************************ EPOCH {epoch} ************************')
    start_time = time.time()  # Start time of the epoch
    model.train()
    for inputs, labels in tqdm(train_loader, desc=f"Epoch: {epoch}"):
      inputs = inputs.to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = model(inputs)
      loss = criterion(nn.functional.softmax(outputs, dim=1), labels)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch
    total_tempo_treinamento += epoch_duration

    print(f'Resultados da época {epoch}')
    print(f"Elapsed Time: {epoch_duration:.2f} sec")
    print_loss_acuracia('[TRAIN]', model, train_loader)
    print_loss_acuracia('[VAL]', model, val_loader)
  print('*'*20)
  print(f'Duração média durante o treino, por época: {total_tempo_treinamento/num_epochs:.2f}')

In [ ]:
inicializa_seeds()
model = OneHotMLP(VOCAB_SIZE, HIDDEN_SIZE, RANK, ALPHA, 2)
train_model(model, lr=LR, num_epochs=NUM_EPOCHS)

Total de parâmetros treináveis: 4000602/4020802
[TRAIN] Antes de iniciar o treinamento. Loss: 0.6927644582748413. Acc: 0.51385
************************ EPOCH 1 ************************


Epoch: 1: 100%|██████████| 157/157 [00:01<00:00, 146.35it/s]


Resultados da época 1
Elapsed Time: 1.08 sec
[TRAIN]. Loss: 0.5444812890052796. Acc: 0.82305
[VAL]. Loss: 0.5499884818077088. Acc: 0.8136
************************ EPOCH 2 ************************


Epoch: 2: 100%|██████████| 157/157 [00:01<00:00, 152.93it/s]


Resultados da época 2
Elapsed Time: 1.04 sec
[TRAIN]. Loss: 0.48147224526405336. Acc: 0.84685
[VAL]. Loss: 0.49002674050331113. Acc: 0.837
************************ EPOCH 3 ************************


...

Resultados da época 5
Elapsed Time: 0.93 sec
[TRAIN]. Loss: 0.4206233283996582. Acc: 0.90465
[VAL]. Loss: 0.44712929768562315. Acc: 0.8708
************************ EPOCH 6 ************************


Epoch: 6: 100%|██████████| 157/157 [00:00<00:00, 160.34it/s]


Resultados da época 6
Elapsed Time: 0.98 sec
[TRAIN]. Loss: 0.41281407041549684. Acc: 0.91285
[VAL]. Loss: 0.44706996631622314. Acc: 0.8654
************************ EPOCH 7 ************************


Epoch: 7: 100%|██████████| 157/157 [00:00<00:00, 171.79it/s]


Resultados da época 7
Elapsed Time: 0.92 sec
[TRAIN]. Loss: 0.4009817371368408. Acc: 0.9268
[VAL]. Loss: 0.43852525539398196. Acc: 0.8754
********************
Duração média durante o treino, por época: 0.98


Com esses resultados, dá pra imaginar que é possível ter aproximadamente 87~88% de acurácia na base de testes:

In [ ]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.4384763907623291. Acc: 0.87576


(0.4384763907623291, 0.87576)

## Testando LoRA

Vamos fazer o seguinte:

1. Reiniciar a seed e reinicializar o modelo.
2. Simular ele completo com apenas uma época => Esse será o modelo pré-treinado (tem uns 78% de acurácia no conjunto de validação => esse valor inicial irá mudar dependendo da inicialização do modelo, mas basta que ele seja alguns pontos abaixo do target de 87-88% pra testarmos)
3. Ativar as camadas LoRA.
4. Simular novamente e verificar melhorias no tempo e se é possível chegar numa acurácia próxima a de treinar o modelo completo.

PASSOS 1 e 2. Reiniciar a seed, reinicializar o modelo, simular com uma época

In [ ]:
inicializa_seeds()
model = OneHotMLP(VOCAB_SIZE, HIDDEN_SIZE, RANK, ALPHA, 2)
train_model(model, lr=LR, num_epochs=1)

Total de parâmetros treináveis: 4000602/4020802
[TRAIN] Antes de iniciar o treinamento. Loss: 0.6927644582748413. Acc: 0.51385
************************ EPOCH 1 ************************


Epoch: 1: 100%|██████████| 157/157 [00:00<00:00, 166.92it/s]


Resultados da época 1
Elapsed Time: 0.95 sec
[TRAIN]. Loss: 0.5444812890052796. Acc: 0.82305
[VAL]. Loss: 0.5499884818077088. Acc: 0.8136
********************
Duração média durante o treino, por época: 0.95


In [ ]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.5512490636062622. Acc: 0.81256


(0.5512490636062622, 0.81256)

PASSOS 3 e 4. Ativar as camadas LoRA e simular

Mas antes de ativar, vamos ver como estão os pesos da camada fc1 do modelo (weight e bias). Depois de treinar os pesos devem ficar iguais. Vamos checar também as matrizes A e B.

In [ ]:
# Guarda para comparar depois
fc1_0 = model.fc1.weight[0, 1:10].detach().clone()
fc1_1 = model.fc1.weight[1, 1:10].detach().clone()
b_0 = model.fc1.bias[1:10].detach().clone()

print('w[0, 1:10]', model.fc1.weight[0, 1:10], '\n')
print('w[1, 1:10]', model.fc1.weight[1, 1:10], '\n')
print('b[1:10]', model.fc1.bias[1:10], '\n')
print('A[0: 1:10]', model.fc1.A[0, 1:10], '\n') # Inicialização gausiana, média 0
print('A[1: 1:10]', model.fc1.A[1, 1:10], '\n') # Inicialização gausiana, média 0
print('B[0: 1:10]', model.fc1.B[0, 1:10], '\n') # Tem que ter só zeros aqui

w[0, 1:10] tensor([-0.0018, -0.0030, -0.0058,  0.0035,  0.0066, -0.0022, -0.0037,  0.0076,
        -0.0119], device='cuda:0', grad_fn=<SliceBackward0>) 

w[1, 1:10] tensor([ 0.0075,  0.0194,  0.0114, -0.0156,  0.0159, -0.0014,  0.0096, -0.0025,
        -0.0152], device='cuda:0', grad_fn=<SliceBackward0>) 

b[1:10] tensor([ 3.5952e-03, -4.8239e-04,  2.0265e-03,  3.6117e-03,  4.5666e-03,
         1.0681e-03,  1.7175e-03,  3.7140e-03, -4.0021e-05], device='cuda:0',
       grad_fn=<SliceBackward0>) 

A[0: 1:10] tensor([], device='cuda:0') 

A[1: 1:10] tensor([], device='cuda:0') 

B[0: 1:10] tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') 



In [ ]:
model.toggle_lora(True)
train_model(model, lr=LR, num_epochs=NUM_EPOCHS_LORA)

Total de parâmetros treináveis: 20602/4020802
[TRAIN] Antes de iniciar o treinamento. Loss: 0.544481286239624. Acc: 0.82305
************************ EPOCH 1 ************************


Epoch: 1: 100%|██████████| 157/157 [00:00<00:00, 164.07it/s]


Resultados da época 1
Elapsed Time: 0.97 sec
[TRAIN]. Loss: 0.5190781641960144. Acc: 0.7954
[VAL]. Loss: 0.5323013854026795. Acc: 0.7778
************************ EPOCH 2 ************************


Epoch: 2: 100%|██████████| 157/157 [00:00<00:00, 169.74it/s]


Resultados da época 2
Elapsed Time: 0.93 sec
[TRAIN]. Loss: 0.46575151739120485. Acc: 0.8414
[VAL]. Loss: 0.48098870248794556. Acc: 0.8228
************************ EPOCH 3 ************************


...

Resultados da época 8
Elapsed Time: 0.97 sec
[TRAIN]. Loss: 0.43763634238243104. Acc: 0.8659
[VAL]. Loss: 0.48059727602005003. Acc: 0.8218
************************ EPOCH 9 ************************


Epoch: 9: 100%|██████████| 157/157 [00:00<00:00, 166.75it/s]


Resultados da época 9
Elapsed Time: 0.95 sec
[TRAIN]. Loss: 0.36940426173210145. Acc: 0.9444
[VAL]. Loss: 0.43479630875587466. Acc: 0.8736
************************ EPOCH 10 ************************


Epoch: 10: 100%|██████████| 157/157 [00:01<00:00, 141.35it/s]


Resultados da época 10
Elapsed Time: 1.12 sec
[TRAIN]. Loss: 0.36321702942848205. Acc: 0.95065
[VAL]. Loss: 0.43189150218963623. Acc: 0.877
********************
Duração média durante o treino, por época: 0.99


In [ ]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.4307401481819153. Acc: 0.87744


(0.4307401481819153, 0.87744)

Checa se os parâmetros weight e bias da camada LoRA mudaram depois do fine-tuning ou se apenas ajustaram as camadas A e B da camada LoRA:

In [ ]:
print(torch.all(fc1_0 == model.fc1.weight[0, 1:10])) # Tem que ser True
print(torch.all(fc1_1 == model.fc1.weight[1, 1:10])) # Tem que ser True
print(torch.all(b_0 == model.fc1.bias[1:10])) # Tem que ser True

print('w[0, 1:10]', model.fc1.weight[0, 1:10], '\n')
print('w[1, 1:10]', model.fc1.weight[1, 1:10], '\n')
print('b[1:10]', model.fc1.bias[1:10], '\n')
print('A[0: 1:10]', model.fc1.A[0, 1:10], '\n') # Nesse ponto essa matriz já foi treinada, tem que ter alguma coisa diferente
print('A[1: 1:10]', model.fc1.A[1, 1:10], '\n') # Nesse ponto essa matriz já foi treinada, tem que ter alguma coisa diferente
print('B[0: 1:10]', model.fc1.B[0, 1:10], '\n') # Nesse ponto essa matriz já foi treinada, não pode continuar zerada

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
w[0, 1:10] tensor([-0.0018, -0.0030, -0.0058,  0.0035,  0.0066, -0.0022, -0.0037,  0.0076,
        -0.0119], device='cuda:0') 

w[1, 1:10] tensor([ 0.0075,  0.0194,  0.0114, -0.0156,  0.0159, -0.0014,  0.0096, -0.0025,
        -0.0152], device='cuda:0') 

b[1:10] tensor([ 3.5952e-03, -4.8239e-04,  2.0265e-03,  3.6117e-03,  4.5666e-03,
         1.0681e-03,  1.7175e-03,  3.7140e-03, -4.0021e-05], device='cuda:0') 

A[0: 1:10] tensor([], device='cuda:0', grad_fn=<SliceBackward0>) 

A[1: 1:10] tensor([], device='cuda:0', grad_fn=<SliceBackward0>) 

B[0: 1:10] tensor([-0.0331,  0.0474,  0.0306,  0.0086,  0.0205, -0.0656,  0.0111, -0.0618,
        -0.0693], device='cuda:0', grad_fn=<SliceBackward0>) 



Transfere a camada LoRA para o modelo (altera weight para scale * A * B):

In [ ]:
model.desabilita_lora_e_transfere_para_modelo()

In [ ]:
print(torch.all(fc1_0 == model.fc1.weight[0, 1:10])) # Tem que ser False
print(torch.all(fc1_1 == model.fc1.weight[1, 1:10])) # Tem que ser False
print(torch.all(b_0 == model.fc1.bias[1:10])) # Tem que ser True

print('w[0, 1:10]', model.fc1.weight[0, 1:10], '\n')
print('w[1, 1:10]', model.fc1.weight[1, 1:10], '\n')
print('b[1:10]', model.fc1.bias[1:10], '\n')
print('A[0: 1:10]', model.fc1.A[0, 1:10], '\n')
print('A[1: 1:10]', model.fc1.A[1, 1:10], '\n')
print('B[0: 1:10]', model.fc1.B[0, 1:10], '\n')

tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
w[0, 1:10] tensor([ 0.0248, -0.0410, -0.0304, -0.0035, -0.0099,  0.0505, -0.0126,  0.0572,
         0.0437], device='cuda:0', grad_fn=<SliceBackward0>) 

w[1, 1:10] tensor([-0.0707,  0.1315,  0.0838,  0.0049,  0.0643, -0.1567,  0.0357, -0.1487,
        -0.1791], device='cuda:0', grad_fn=<SliceBackward0>) 

b[1:10] tensor([ 3.5952e-03, -4.8239e-04,  2.0265e-03,  3.6117e-03,  4.5666e-03,
         1.0681e-03,  1.7175e-03,  3.7140e-03, -4.0021e-05], device='cuda:0',
       grad_fn=<SliceBackward0>) 

A[0: 1:10] tensor([]) 

A[1: 1:10] tensor([]) 

B[0: 1:10] tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.]) 



Agora que já jogou os dados de A e B para weight, calcula novamente a acurácia (deve estar igual o último cálculo)

In [ ]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.4307401481819153. Acc: 0.87744


(0.4307401481819153, 0.87744)